In [3]:
import pandas as pd

In [4]:
signals = pd.read_parquet("signals.parquet.gzip")
metatags = pd.read_parquet("metatags.parquet.gzip")

In [5]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [6]:
articles = intersection(metatags.src.unique(), signals.src.unique() )
signals = signals[signals.src.isin(articles)]
metatags = metatags[metatags.src.isin(articles)]
len(articles)

898

In [7]:
metatags

,src,keywords,themes,summary,origin,url
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...
3,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...
4,58c165e1a46408307f2b60f63df7c549,"[E-seeds, drill, moistened, drones, seeding, b...","[plant-inspired technology, seed burial, agric...",A team from Carnegie Mellon University has dev...,20220225,b'https://newatlas.com/science/erodium-inspire...
...,...,...,...,...,...,...
1056,4d1abdf7e702b559c6ccff847ce4d8d0,"[backdooring, summarizerbot, shape opinion, mo...","[Machine learning, Cybersecurity, Manipulation]",This text discusses the risks associated with ...,20221031,b'https://pluralistic.net/2022/10/21/let-me-su...
1057,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...
1058,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...
1060,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...


In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [10]:
#sentences = list(metatags.summary)
sentences = list(metatags.keywords)
sentences = [", ".join(x) for x in sentences]

In [11]:
from sentence_transformers import util


In [12]:
embeddings = model.encode(sentences, convert_to_tensor=True,show_progress_bar=True)
cosine_scores = util.cos_sim(embeddings, embeddings)
#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

trial, digital access, FT.com, Standard Digital, Premium Digital, global news, analysis, expert opinion, Lex, business themes 		 trial, digital access, FT.com, Standard Digital, Premium Digital, global news, analysis, expert opinion, Lex, business themes 		 Score: 1.0000
keyword0 for keyword in top_keywords 		 keyword0 for keyword in top_keywords 		 Score: 1.0000
counterfeit people, artificial intelligence, immoral act, vandalism, counterfeiting, trust, human freedom, creation, deepfakes, counterfeit digital people 		 counterfeit people, artificial intelligence, immoral act, vandalism, counterfeiting, trust, digital environments, dangerous artifacts, economies, human freedom 		 Score: 0.9684
trial, digital access, FT.com, Standard Digital, Premium Digital, global news, analysis, expert opinion, Lex, business themes 		 trial, digital access, FT.com, Standard Digital, Premium Digital, global news, analysis, expert opinion, business column, curated newsletters 		 Score: 0.9674
trial, digi

It takes 10 mins for 500 articles. Approx.

In [13]:
dfSent = pd.DataFrame([sentences, list(metatags.src)]).T
dfSent.columns = ["txt","src"]
dfSent.to_parquet("pages/dfSentences.parquet.gzip",compression="gzip")
dfSent

,txt,src
0,"AI panic, classrooms, education system, adapt,...",ea74b217d36ec570bb432fb5aa679090
1,"backdoor, cybersecurity researchers, encrypted...",841546c0efc4c82f0aabc545a47a09e1
2,"Cybersecurity Researchers, Sandbox Satellite, ...",f6637d1fa0e3fe0e1e94bea8b3338ef9
3,"Chat-GPT, Python, Knowledge Graph, Neo4j, arti...",7ff6239e13a43e856fa36887902c5ede
4,"E-seeds, drill, moistened, drones, seeding, bi...",58c165e1a46408307f2b60f63df7c549
...,...,...
893,"backdooring, summarizerbot, shape opinion, mod...",4d1abdf7e702b559c6ccff847ce4d8d0
894,"hanging out, book, Sheila Liming, crisis, isol...",edbb7336dfb26098bcc966ca17c074b9
895,"insomniacs, sleeping assistants, Taobao, Chine...",3fbb11d6e949d1e662aa6a146bb6cda0
896,"U.S. government, Nvidia, high-performance comp...",b79a2baa87b68283198416791b93bce4


In [14]:
dfPairs = pd.DataFrame(pairs)
dfPairs["score"] = dfPairs["score"].astype(float)
dfPairs["i"] = dfPairs["index"].apply(lambda x: x[0])
dfPairs["j"] = dfPairs["index"].apply(lambda x: x[1])
dfPairs.to_parquet("pages/dfPairs.parquet.gzip",compression="gzip")
dfPairs

,index,score,i,j
0,"[28, 890]",1.000000,28,890
1,"[336, 811]",1.000000,336,811
2,"[97, 295]",0.968448,97,295
3,"[28, 380]",0.967408,28,380
4,"[380, 890]",0.967408,380,890
...,...,...,...,...
402748,"[96, 806]",-0.040864,96,806
402749,"[247, 250]",-0.040947,247,250
402750,"[310, 791]",-0.050092,310,791
402751,"[405, 791]",-0.056254,405,791


In [15]:
def getClosest(i):
    RES = dfPairs[ ((dfPairs["i"] == i) & (dfPairs["j"] != i)) \
      + ((dfPairs["j"] == i) & (dfPairs["i"] != i))]
    closest = list(RES.sort_values(by="score", ascending=False )["j"][:10])
    return [metatags.iloc[x].src for x in closest]

In [16]:
metatags = metatags.reset_index()
metatags = metatags.drop("index",axis=1)
metatags

,src,keywords,themes,summary,origin,url
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...
3,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...
4,58c165e1a46408307f2b60f63df7c549,"[E-seeds, drill, moistened, drones, seeding, b...","[plant-inspired technology, seed burial, agric...",A team from Carnegie Mellon University has dev...,20220225,b'https://newatlas.com/science/erodium-inspire...
...,...,...,...,...,...,...
893,4d1abdf7e702b559c6ccff847ce4d8d0,"[backdooring, summarizerbot, shape opinion, mo...","[Machine learning, Cybersecurity, Manipulation]",This text discusses the risks associated with ...,20221031,b'https://pluralistic.net/2022/10/21/let-me-su...
894,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...
895,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...
896,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...


In [35]:
metatags["closest"] = ""
for ix, row in metatags.iterrows():
    row["closest"] = getClosest(ix)
metatags

/tmp/ipykernel_26163/2091420612.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  closest = list(RES.sort_values(by="score", ascending=False )["j"][:10])


,src,keywords,themes,summary,origin,url,closest,kwsl
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...,"[b30a4282af9e53ca673438a8223d9525, 5a06b540b77...","[ai panic, classrooms, education system, adapt..."
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...,"[f6637d1fa0e3fe0e1e94bea8b3338ef9, ee0bcad312f...","[backdoor, cybersecurity researchers, encrypte..."
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...,"[ae919b08f9b58de655abd2fea03c1169, 0b6d917158c...","[cybersecurity researchers, sandbox satellite,..."
3,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...,"[e0f6ae75e034df0a32044fe8a9075673, f43094a456d...","[chat-gpt, python, knowledge graph, neo4j, art..."
4,58c165e1a46408307f2b60f63df7c549,"[E-seeds, drill, moistened, drones, seeding, b...","[plant-inspired technology, seed burial, agric...",A team from Carnegie Mellon University has dev...,20220225,b'https://newatlas.com/science/erodium-inspire...,"[d812f238b024d5a104cbb7c001ed400b, 58b1d428223...","[e-seeds, drill, moistened, drones, seeding, b..."
...,...,...,...,...,...,...,...,...
893,4d1abdf7e702b559c6ccff847ce4d8d0,"[backdooring, summarizerbot, shape opinion, mo...","[Machine learning, Cybersecurity, Manipulation]",This text discusses the risks associated with ...,20221031,b'https://pluralistic.net/2022/10/21/let-me-su...,"[4d1abdf7e702b559c6ccff847ce4d8d0, 4d1abdf7e70...","[backdooring, summarizerbot, shape opinion, mo..."
894,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...,"[edbb7336dfb26098bcc966ca17c074b9, edbb7336dfb...","[hanging out, book, sheila liming, crisis, iso..."
895,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...,"[3fbb11d6e949d1e662aa6a146bb6cda0, 3fbb11d6e94...","[insomniacs, sleeping assistants, taobao, chin..."
896,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...,"[b79a2baa87b68283198416791b93bce4, b79a2baa87b...","[u.s. government, nvidia, high-performance com..."


In [36]:
metatags.to_parquet("pages/metatags_with_closest.parquet.gzip", compression="gzip")

In [37]:
metatags = pd.read_parquet("pages/metatags_with_closest.parquet.gzip")

In [38]:
signals

,Signal,Change,10y horizon,Driving force,src
0,AI's impact on education ...,Adaptation and improvement,"Integration of AI in classrooms, AI tutoring,...",Potential for improved learning and teaching ...,ea74b217d36ec570bb432fb5aa679090
1,AI cheating in classrooms ...,Widespread and undetectable,"Continued AI cheating, advancements in AI tec...",Easier access to AI technology and the desire...,ea74b217d36ec570bb432fb5aa679090
2,AI transforming classrooms ...,Fundamental transformation,"Flipped classrooms, personalized learning, en...",Potential to enhance learning experience and ...,ea74b217d36ec570bb432fb5aa679090
3,Concerns about the impact of calculators on e...,Transition and adaptation,"Calculators integrated into curriculum, chang...",Desire for increased student engagement and p...,ea74b217d36ec570bb432fb5aa679090
4,Potential impact of AI on literacy skills ...,Uncertain,"Potential decline in literacy skills, relianc...",Potential concern for declining writing skill...,ea74b217d36ec570bb432fb5aa679090
...,...,...,...,...,...
5803,Middle East countries impacted by restrictions,Limitation on access to high-performance GPUs,Shift in AI technology supply chains ...,Concerns over technology diversion to China ...,b79a2baa87b68283198416791b93bce4
5804,Saudi Arabia and UAE strengthening AI prowess,Increased focus on AI development,Greater investment and collaboration in AI ...,Desire to advance AI capabilities ...,b79a2baa87b68283198416791b93bce4
5805,U.S. concern over technology diversion to China,Efforts to prevent Chinese AI development,Stricter regulations and controls on technology,Protecting domestic AI industry and capabilit...,b79a2baa87b68283198416791b93bce4
5806,Nvidia working with U.S. government,Collaboration to address licensing issue,Resolution of licensing requirements ...,Maintaining business operations and revenue s...,b79a2baa87b68283198416791b93bce4


In [39]:
titles = pd.read_parquet("titles.parquet.gzip")
titles
TITLES = {}
for ix, row in titles.iterrows():
    TITLES[row["src"]] =  row["title"] 

In [40]:
metatags["kwsl"] = metatags["keywords"].apply(lambda lst: [x.lower().replace('"','') for x in lst])


In [41]:
for ix, row in metatags.iterrows():
    ID = row["src"]
    TXT = "# __"+TITLES[ID] +"__, from (["+row.origin+"](https://kghosh.substack.com/p/"+row.origin+").)\n\n"
    TXT += "__[External link]("+str(row.url)[2:-1]+")__\n\n"
    TXT += "\n\n## Summary\n\n"
    TXT += row["summary"] + "\n\n## Keywords\n\n"
    TXT += "* "+"\n* ".join(row["keywords"])
    TXT += "\n\n## Themes\n\n"
    TXT += "* "+"\n* ".join(row["themes"])
    TXT += "\n\n## Signals\n\n"    
    SIG = signals[signals.src == ID]
    TXT += SIG[list(SIG.columns)[:-1]].to_markdown(index=False)
    TXT += "\n\n## Closest\n\n"
    TXT += "* "+"\n* ".join([ "["+TITLES[x]+"]("+x+")" for x in row["closest"]])
    with open("docs/"+ID+".md", "w") as f:
        f.write(TXT) 

    

In [42]:
metatags

,src,keywords,themes,summary,origin,url,closest,kwsl
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...,"[b30a4282af9e53ca673438a8223d9525, 5a06b540b77...","[ai panic, classrooms, education system, adapt..."
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...,"[f6637d1fa0e3fe0e1e94bea8b3338ef9, ee0bcad312f...","[backdoor, cybersecurity researchers, encrypte..."
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...,"[ae919b08f9b58de655abd2fea03c1169, 0b6d917158c...","[cybersecurity researchers, sandbox satellite,..."
3,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...,"[e0f6ae75e034df0a32044fe8a9075673, f43094a456d...","[chat-gpt, python, knowledge graph, neo4j, art..."
4,58c165e1a46408307f2b60f63df7c549,"[E-seeds, drill, moistened, drones, seeding, b...","[plant-inspired technology, seed burial, agric...",A team from Carnegie Mellon University has dev...,20220225,b'https://newatlas.com/science/erodium-inspire...,"[d812f238b024d5a104cbb7c001ed400b, 58b1d428223...","[e-seeds, drill, moistened, drones, seeding, b..."
...,...,...,...,...,...,...,...,...
893,4d1abdf7e702b559c6ccff847ce4d8d0,"[backdooring, summarizerbot, shape opinion, mo...","[Machine learning, Cybersecurity, Manipulation]",This text discusses the risks associated with ...,20221031,b'https://pluralistic.net/2022/10/21/let-me-su...,"[4d1abdf7e702b559c6ccff847ce4d8d0, 4d1abdf7e70...","[backdooring, summarizerbot, shape opinion, mo..."
894,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...,"[edbb7336dfb26098bcc966ca17c074b9, edbb7336dfb...","[hanging out, book, sheila liming, crisis, iso..."
895,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...,"[3fbb11d6e949d1e662aa6a146bb6cda0, 3fbb11d6e94...","[insomniacs, sleeping assistants, taobao, chin..."
896,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...,"[b79a2baa87b68283198416791b93bce4, b79a2baa87b...","[u.s. government, nvidia, high-performance com..."


In [43]:
allkw = []
for kw in metatags.themes:# keywords: #as an alternative
    allkw = allkw+list(kw)
print(len(allkw))
KW = list(set(allkw))
KW = list(set([x.lower().replace('"','') for x in KW]))
KW = [x for x in KW if not x.isnumeric()]
KW.sort()
print(len(KW))
KW

2710
2276


['2024 problem in logistics',
 '3d imaging',
 '3d printing technology in construction',
 'academic vs. intellectual',
 'access plans',
 'access to culture',
 'access to fresh groceries',
 'access to tools and resources',
 'accessibility and applications of ai to everyday life',
 'accountability',
 'accounting',
 'activation',
 'activism',
 'activisme écologique',
 'activity partners',
 'adaptation of education system',
 'adapting to ai in the classroom',
 'adaptive reuse apartments',
 'adding',
 'adhd in the workplace',
 'advanced bionic limb',
 'advanced technology',
 'advancements in ai',
 'advancements in civil engineering',
 'advancements in generative ai',
 'advancements in watch technology',
 'advances in artificial intelligence',
 'advances in language models',
 'advances in medical imaging',
 'advantages of ai-based animations',
 'advantages of chatgpt in agile coaching',
 'advantages of gen z in navigating software',
 'advantages of magnesium as a structural metal',
 'adventur

In [44]:
import string
LETTERS = string.ascii_lowercase
LETTERS

'abcdefghijklmnopqrstuvwxyz'

In [48]:
INDEX = "Index \n\n"

for letter in LETTERS:
    kws = [x for x in KW if x[0] == letter]
    if len(kws):
        INDEX += "\n\n## Letter '"+letter.upper()+"' \n\n"
        for kw in kws:
            INDEX += "\n* __"+kw+"__: "
            mask = metatags.themes.apply(lambda x: any(item for item in [kw] if item in x))
            lst = metatags[mask].src.tolist()
            INDEX += ", ".join([ "["+TITLES[x]+"]("+x+")" for x in lst ])
INDEX += "## Others\n\n"
kws = [x for x in KW if x[0].isnumeric()]
for kw in kws:
    INDEX += "\n* __"+kw+"__: "
    mask = metatags.kwsl.apply(lambda x: any(item for item in [kw] if item in x))
    lst = metatags[mask].src.tolist()
    INDEX += ", ".join([ "["+TITLES[x]+"]("+x+")" for x in lst ])

In [49]:
with open("docs/index.md", "w") as f:
    f.write(INDEX) 

In [50]:
INDEX

'Index \n\n\n\n## Letter \'A\' \n\n\n* __academic vs. intellectual__: [The Difference Between Academics and Intellectuals](f07df9d85e95d88e18345a39166c0a53)\n* __access plans__: [FT.com Trial and Subscription Information](34e1369572f1241d4a54e63cee2a4565)\n* __access to culture__: \n* __access to fresh groceries__: \n* __access to tools and resources__: [American Farm Bureau and John Deere Sign Agreement on Equipment Repair](26ae831dfd3721235ac0eacb85e4a0e5)\n* __accessibility and applications of ai to everyday life__: \n* __accountability__: \n* __accounting__: [The Triumph of the Bean Counters](74640ec9669a4ca9964b6a3aca93f336)\n* __activation__: [Prada Seeds: Blooming Luxury with Fall Flowers](477eb369e45ddd2deab9a1d75fd105ec)\n* __activism__: [Hope in the Dark: A Radical Case for Hope in Uncertain Times](fd193cbb838e7049c4d44c7dde929f83)\n* __activisme écologique__: [Young Activists Express Climate Concerns](a33285a958e1c67e274803ca7ce0bec2)\n* __activity partners__: \n* __adaptati